In [9]:
!pip install scikit-build-core==0.9.0

  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 13.1 MB/s eta 0:00:00
Using cached pathspec-0.12.1-py3-none-any.whl (31 kB)


In [11]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=61" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.62 --force-reinstall --upgrade --no-cache-dir --verbose --no-build-isolation

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 16.5 MB/s eta 0:00:00
  Running command Preparing metadata (pyproject.toml)
  *** scikit-build-core 0.9.0 using CMake 3.30.2 (metadata_wheel)
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for typing-extensions>=4.5.0 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for numpy>=1.20.0 from https://files.pythonhosted.org/packages/7d/4b/a509d346fffede6120cc17610cc500819417ee9c3da7f08d9aaf15cab2a3/numpy-2.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 35.9 MB/s eta 0:00:00
  Obtaining dependency information for diskcache>=5.6.1 from https://files.pythonhosted.org/packages/3f/27/4570e78fc0bf

In [3]:
!wget https://huggingface.co/fhai50032/BeagleLake-7B-GGUF/resolve/main/BeagleLake-7B.Q8_0.gguf

--2024-09-05 18:15:22--  https://huggingface.co/fhai50032/BeagleLake-7B-GGUF/resolve/main/BeagleLake-7B.Q8_0.gguf
Resolving huggingface.co (huggingface.co)... 13.33.28.94, 13.33.28.87, 13.33.28.54, ...
Connecting to huggingface.co (huggingface.co)|13.33.28.94|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/f0/05/f005d362adc1529258f3fc67c74b31622c8d74747cc9188de7811a7dcf1d7339/b629064f3f5562a7746f2f83602c7ab952b8c7ade62d8335cba828e0fd7df0bc?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27BeagleLake-7B.Q8_0.gguf%3B+filename%3D%22BeagleLake-7B.Q8_0.gguf%22%3B&Expires=1725819322&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNTgxOTMyMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2YwLzA1L2YwMDVkMzYyYWRjMTUyOTI1OGYzZmM2N2M3NGIzMTYyMmM4ZDc0NzQ3Y2M5MTg4ZGU3ODExYTdkY2YxZDczMzkvYjYyOTA2NGYzZjU1NjJhNzc0NmYyZjgzNjAyYzdhYjk1MmI4YzdhZGU2MmQ4

In [4]:
!pip install pyngrok --quiet
!pip install starlette-context

In [1]:
import os
from pyngrok import ngrok, conf
os.environ["NGROK"] = "2bWuFf5hFuuv761QzAJBnNAiAq9_4tCnQseBH6hQBCYrQrkij"
conf.get_default().auth_token = os.environ["NGROK"]

In [2]:
import json
port=911
config_content = {
    "host": "0.0.0.0",
    "port": f"{port}",
    "models": [
        {
            "model": "/content/BeagleLake-7B.Q8_0.gguf",
            "model_alias": "gpt-3.5-turbo",
            "chat_format": "chatml",
            "n_gpu_layers": -1,
            "offload_kqv": True,
            "n_threads": 12,
            "n_batch": 512,
            "n_ctx": 20006
        }]
}

with open('config.json', 'w') as json_file:
    json.dump(config_content, json_file, indent=4)


In [3]:
!python3 -m llama_cpp.server --config_file /content/config.json > server.log 2>&1 &

In [4]:
public_url = ngrok.connect(f"{port}").public_url
print("ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

ngrok tunnel "https://ee01-34-143-169-105.ngrok-free.app" -> "http://127.0.0.1:911"


In [ ]:
!pkill uvicorn

In [21]:
import gc
gc.collect()
import torch
# if llm:
#   del llm
# llm=None
torch.cuda.empty_cache()